In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('var_apl.xlsx', sheet_name=1)
df = df.set_index('Date')
df = df.sort_index()
df.head()

,Bid,Ask,Bid.1,Ask.1
Date,,,,
2022-03-03,127.546875,127.56250,156.87500,156.90625
2022-03-04,128.421880,128.46875,159.00000,159.06250
2022-03-07,127.968750,128.00000,158.84375,158.87500
2022-03-08,127.328125,127.34375,157.46875,157.50000
2022-03-09,126.515625,126.53125,155.25000,155.28125


In [5]:
# Bid y Ask
bid_columns = [col for col in df.columns if 'Bid' in col]
ask_columns = [col for col in df.columns if 'Ask' in col]

# Mid
mid_columns = [f'Mid.{i}' for i in range(len(bid_columns))]
df[mid_columns] = (df[bid_columns].values + df[ask_columns].values) / 2

# Spreads
spread_columns = [f'Spread.{i}' for i in range(len(bid_columns))]
df[spread_columns] = (df[ask_columns].values - df[bid_columns].values) / df[mid_columns].values

# Returns
return_columns = [f'Return.{i}' for i in range(len(mid_columns))]
df[return_columns] = df[mid_columns].pct_change()

posiciones = [100000000, 90000000]
value = posiciones * df[mid_columns].iloc[-1].values
pv = np.sum(value)
w = value / pv

df['port_ret'] = np.dot(df[return_columns], w)
df

,Bid,Ask,Bid.1,Ask.1,Mid.0,Mid.1,Spread.0,Spread.1,Return.0,Return.1,port_ret
Date,,,,,,,,,,,
2022-03-03,127.546875,127.562500,156.87500,156.90625,127.554688,156.890625,0.000122,0.000199,NaN,NaN,NaN
2022-03-04,128.421880,128.468750,159.00000,159.06250,128.445315,159.031250,0.000365,0.000393,0.006982,0.013644,0.010247
2022-03-07,127.968750,128.000000,158.84375,158.87500,127.984375,158.859375,0.000244,0.000197,-0.003589,-0.001081,-0.002360
2022-03-08,127.328125,127.343750,157.46875,157.50000,127.335938,157.484375,0.000123,0.000198,-0.005067,-0.008655,-0.006825
2022-03-09,126.515625,126.531250,155.25000,155.28125,126.523438,155.265625,0.000123,0.000201,-0.006381,-0.014089,-0.010158
...,...,...,...,...,...,...,...,...,...,...,...
2025-02-25,110.515625,110.531250,117.62500,117.65625,110.523438,117.640625,0.000141,0.000266,0.006403,0.012098,0.009194
2025-02-26,110.765625,110.796875,118.18750,118.21875,110.781250,118.203125,0.000282,0.000264,0.002333,0.004782,0.003533
2025-02-27,110.812500,110.828125,117.78125,117.81250,110.820312,117.796875,0.000141,0.000265,0.000353,-0.003437,-0.001504
